In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import os

c:\Users\Balas\anaconda3\envs\imganenv\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
data_dir = "D:\\Dataset_Crawl\\processed_images"

In [3]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 16

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [4]:
train_data = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset='training',
    class_mode='categorical'
)

Found 5489 images belonging to 10 classes.


In [5]:
val_data = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset='validation',
    class_mode='categorical'
)


Found 1369 images belonging to 10 classes.


In [6]:
#Base_model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(train_data.num_classes, activation='softmax')(x)

In [7]:
model = Model(inputs=base_model.input, outputs=output)

In [8]:
for layer in base_model.layers:
    layer.trainable = False


In [9]:
from tensorflow.keras.callbacks import EarlyStopping

# Early stopping
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(
    train_data,
    validation_data=val_data,
    epochs=30,
    callbacks=[early_stop]
)



Epoch 1/30
344/344 [==============================] - 69s 197ms/step - loss: 0.8261 - accuracy: 0.7311 - val_loss: 0.8803 - val_accuracy: 0.7129
Epoch 2/30
344/344 [==============================] - 55s 160ms/step - loss: 0.3379 - accuracy: 0.8892 - val_loss: 0.6594 - val_accuracy: 0.7882
Epoch 3/30
344/344 [==============================] - 55s 161ms/step - loss: 0.2009 - accuracy: 0.9368 - val_loss: 0.6361 - val_accuracy: 0.7984
Epoch 4/30
344/344 [==============================] - 55s 159ms/step - loss: 0.1116 - accuracy: 0.9679 - val_loss: 0.7093 - val_accuracy: 0.8035
Epoch 5/30
344/344 [==============================] - 56s 162ms/step - loss: 0.0759 - accuracy: 0.9807 - val_loss: 0.6938 - val_accuracy: 0.7918
Epoch 6/30
344/344 [==============================] - 56s 162ms/step - loss: 0.0391 - accuracy: 0.9925 - val_loss: 0.6762 - val_accuracy: 0.8035


In [10]:
model.save("issue_classifier.h5")

c:\Users\Balas\anaconda3\envs\imganenv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
print(train_data.class_indices)



{'Road_work': 0, 'School_damage': 1, 'Sewage_and_Water_Stagnation': 2, 'Street_Light_Issue': 3, 'Toilet_Issue': 4, 'garbage': 5, 'not_maintained_parks': 6, 'road_damage': 7, 'shop_obstructing_pathway': 8, 'tree_obstructing_road': 9}
